In [1]:
%load_ext dotenv
%dotenv

In [1]:
%%md
* EMMA - cex API websockets
* Replicate functionality REST and websocket
* Submit order REST and hear about it via websocket for L2 updates
* Orders that are placed, separate positions call

* L2 updates on websocket
* websocket feed for trade fills (personal only). DYDX pending, open, cancelled
* Submit over REST

UsageError: Cell magic `%%md` not found.


In [1]:
import logging
import os
from zeta_py.client import Client
from zeta_py.types import Asset, Side
import anchorpy
from solana.rpc.commitment import Confirmed
from datetime import datetime, timedelta
from zeta_py.types import OrderOptions, OrderType
from solana.rpc.types import TxOpts
from datetime import datetime, timedelta
from zeta_py.types import OrderOptions, OrderType
from zeta_py import utils

logger = logging.getLogger("zeta_py")
logging.basicConfig(level=logging.DEBUG)

endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
print(endpoint.split(".com")[0])

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet
print(wallet.public_key)

https://api.mainnet-beta.solana
4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh


In [2]:
# print(f"Balance: {(await conn.get_balance(wallet.public_key)).value / 1e9} SOL")

In [3]:
tx_opts = TxOpts({"preflight_commitment": Confirmed, "skip_confirmation": False})

asset = Asset.SOL
client = await Client.load(endpoint=endpoint, commitment=Confirmed, wallet=wallet, assets=[asset], tx_opts=tx_opts)

In [4]:
balance = await client.fetch_balance()
position = await client.fetch_position(asset)
open_orders = await client.fetch_open_orders(asset)

print(f"Balance: {balance}")
print(f"Position: {position}")
print(f"Open orders: {open_orders}")

Balance: 0.489298
Position: Position(size=0.003, cost_of_trades=0.058236)
Open orders: []


In [5]:
from zeta_py.orderbook import Orderbook


async def orderbook_callback(orderbook: Orderbook):
    print(orderbook._get_l2(5))


await client.subscribe_orderbook(asset, Side.Bid, orderbook_callback)

INFO:zeta_py.client.Client:Subscribing to Orderbook:Side.Bid.


[OrderInfo(price=18.0685, size=276.926), OrderInfo(price=18.0649, size=1661.555), OrderInfo(price=18.06, size=27.68), OrderInfo(price=18.0576, size=3323.11), OrderInfo(price=18.055, size=138.41)]
[OrderInfo(price=18.0708, size=11.077), OrderInfo(price=18.069, size=276.927), OrderInfo(price=18.067, size=27.67), OrderInfo(price=18.0654, size=1661.563), OrderInfo(price=18.062, size=138.35)]
[OrderInfo(price=18.0708, size=11.077), OrderInfo(price=18.069, size=276.927), OrderInfo(price=18.067, size=27.67), OrderInfo(price=18.0654, size=1661.563), OrderInfo(price=18.062, size=138.35)]
[OrderInfo(price=18.0708, size=11.077), OrderInfo(price=18.069, size=276.927), OrderInfo(price=18.067, size=27.67), OrderInfo(price=18.0654, size=1661.563), OrderInfo(price=18.062, size=138.35)]
[OrderInfo(price=18.0706, size=11.078), OrderInfo(price=18.0688, size=276.937), OrderInfo(price=18.067, size=27.67), OrderInfo(price=18.0652, size=1661.624), OrderInfo(price=18.062, size=138.35)]
[OrderInfo(price=18.070

INFO:zeta_py.client.Client:WebSocket subscription task cancelled.


In [6]:
await client.deposit(0.1)

INFO:zeta_py.client.Client:Depositing 0.1 USDC to margin account


Signature(
    3qxbAKSMXw3RgNufMLD2iUYUsQ2xHpw1YcrSGjg6JuXigoXWTb4g57fVuL4ybzCm5ksc2vuwMNyWxXCDhuxSYSyC,
)

In [6]:
# Place order

expiry_ts = int((datetime.now() + timedelta(seconds=1000)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.PostOnlySlide, client_order_id=69)
await client.place_order(Asset.SOL, 0.1, 0.001, Side.Ask, order_opts)

INFO:zeta_py.client.Client:Placed 0.001x SOL-PERP Ask @ $0.1


Signature(
    4eLF5qiQEJALujpo8WbDwLqAUGa7seJCoA9sWvB61XqQjNQsgwPF4tx7QTLvtxDp3d7JvvXkZWW3wqCZG94HPMQy,
)

In [7]:
open_orders = await client.fetch_open_orders(Asset.SOL)
print("Open Orders:")
for order in open_orders:
    print(f"{order.side.name} {order.info.size}x ${order.info.price}")

Open Orders:
Ask 0.001x $18.0595


In [9]:
oid = open_orders[0].order_id
await client.cancel_order(Asset.SOL, order_id=oid, side=Side.Ask)

INFO:zeta_py.client.Client:Cancelling order 333138974599157647524669740 for SOL


Signature(
    63pQEFpwQrQ41bitfRx3tADDyCkwx5qzec2gs3ivtN6HZR4eHbqmgXqR7eiCWawxj8uG3Pd5ppCmGVFSrq5yDaPF,
)

# Quoting
Use this code to atomically cancel and replace a set of quotes on Zeta.

*For this example we quote 0.001 SOL-PERP contracts at 20bps spread*

In [10]:
# Quoting parameters
asset = Asset.SOL  # SOL-PERP market
quote_size = 0.001  # min size
quote_bps = 10  # basis points from midpoint
time_in_force = 10  # expire the quote after 10 seconds

# Get the latest mark price and calculate bid/ask prices
price_fixed = client.exchange.pricing.mark_prices[asset.to_index()]
bid_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 - quote_bps / 10000))
ask_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 + quote_bps / 10000))

# Set order options
expiry_ts = int((datetime.now() + timedelta(seconds=time_in_force)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.Limit, client_order_id=69)

# Execute quote!
await client.replace_quote(Asset.SOL, bid_price, quote_size, ask_price, quote_size, order_opts)

INFO:zeta_py.client.Client:Replacing SOL orders: 0.001x Bid @ $18.071849061, 0.001x Ask @ $18.108028939


Signature(
    4RJWcc6zEp7ojQthxwarr5C9xTzbo19b1r7pciM4o7LV1Pe5hTTkSuinHzESsgFmWd28RDmWiNkysByRk7BQEaH6,
)